# Blackjack with AI player

In [265]:
import random

In [266]:
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, rank = None):
        self.rank = rank
        
        if not rank: # null card
            self.value = 0
            self.isAce = False
            return
        elif rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
         
    def __str__(self):
        return str(self.rank)
    
    def __eq__(self, card):
        return self.value == card.value


# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.cards = []
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        for i in range(nDecks):
            for j in range(4): # 4 of each card per deck
                for rank in RANKS:
                    self.cards.append(Card(rank))
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.bet = bet
        self.cards = []
        if card:
            self.cards.append(card)
        
    def addCard(self, card):
        self.cards.append(card)     
            
    def removeCard(self):
        return self.cards.pop()
    
    def isBust(self):
        if self.value > 21:
            return True
        return False
    
    def isBlackjack(self):
        if len(self.cards) == 2:
            if self.value == 21:
                return True
        return False
        
    def calculateValue(self, prop = 'value'):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        if (prop == 'aces'):
            return aces
        return value
        
    @property
    def value(self):
        return self.calculateValue()
    
    @property
    def soft(self):
        if self.calculateValue('aces'):
            return True
        return False
              
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
        

#### Testing Card, Deck, and Hand

In [267]:
ace = Card('A')
king = Card('K')
queen = Card('Q')
jack = Card('J')
ten = Card('10')
nine = Card('9')
eight = Card('8')
seven = Card('7')
six = Card('6')
five = Card('5')
four = Card('4')
three = Card('3')
two = Card('2')
noCard = Card()
passed = 0

# Test A,K,Q,9,4
if ace.value == 11:
    if ace.rank == 'A':
        passed += 1
if king.value == 10:
    if king.rank == 'K':
        passed += 1
if queen.value == 10:
    if queen.rank == 'Q':
        passed += 1
if four.value == 4:
    if four.rank == '4':
        passed += 1
if nine.value == 9:
    if nine.rank == '9':
        passed += 1
if noCard.value == 0:
        passed += 1

# Test Equality and Inequality
if king == queen:
    passed += 1 
if not king == nine:
    passed += 1
    
# All Tests
if passed == 8:
    print('All tests passed')
else:
    print(str(8-passed) + ' tests failed')

All tests passed


In [268]:
passed = 0
deck = Deck(1)

# Testing creating a deck and drawing cards from the deck
if len(deck) == 52:
    passed += 1
for i in range(5):
    deck.draw()
if len(deck) == 47:
    passed +=1
deck = Deck(6)
if len(deck) == 312:
    passed += 1
if passed == 3:
    print('All tests passed')
else:
    print(str(3-passed) + ' tests failed')

All tests passed


In [269]:
hand = Hand()
passed = 0

# Test empty hand
if len(hand) == 0:
    passed += 1
    
# Test hand.addCard()
hand.addCard(ace)
if len(hand) == 1:
    passed += 1
hand.addCard(nine)
if len(hand) == 2:
    passed += 1
    
# Test hand.isBust()
if not hand.isBust(): # no bust
    passed += 1
hand.addCard(king)
if not hand.isBust(): # no bust
    passed += 1
hand.addCard(ace)
hand.addCard(ace)
if hand.isBust():
    passed += 1

# Test hand.isBlackjack()
if not hand.isBlackjack(): # no Blackjack
    passed += 1
hand = Hand(ace)
hand.addCard(king)
if hand.isBlackjack(): # Blackjack
    passed += 1
hand = Hand(king)
hand.addCard(king)
if not hand.isBlackjack(): # no blackjack
    passed += 1
       
# Test hand.removeCard and hand.value
hand.removeCard()
hand.addCard(nine)
if hand.value == 19:
    passed += 1

if passed == 10:
    print('All tests passed')
else:
    print(str(10-passed) + ' tests failed')

All tests passed


## Player and Dealer

In [270]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000, bet = 50):
        self.money = money
        self.betAmount = bet
        self.bets = []
        self.hands = [Hand()]
        
    def makeBet(self, handNumber):
        if handNumber >= len(self.bets): # new hand gets a new bet
            self.bets.append(self.betAmount)
        else: # double the bet
            self.bets[handNumber] += self.betAmount
        # remove the bet amount from players money
        self.money -= self.betAmount
            
    # True if player has enough money to make a bet, False otherwise
    def hasMoney(self):
        return self.money > self.betAmount
        
    def hit(self, handNumber, card):
        self.hands[handNumber].addCard(card)

    def doubleDown(self, handNumber, card):
        self.hit(handNumber, card)
        self.makeBet(handNumber)

    def split(self, handNumber, cards):
        newHand = Hand()
        # remove one card from the hand being split and add to newHand
        newHand.addCard(self.hands[handNumber].removeCard())
        # add a second card to the old hand
        self.hands[handNumber].addCard(cards[0])  
        # add a second card to the new hand
        newHand.addCard(cards[1])
        # append newHand to players Hands and add bet for the hand
        self.hands.append(newHand)
        self.makeBet(len(self.hands)-1) # make a new bet for the new hand
        
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print()        

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            self.player.hands[0].addCard(self.deck.draw()) # Deal the player a card   
        self.player.makeBet(0) # player makes a bet on hand
       
    def play(self):
        while self.hand.value <= 17: # hits until they have 17
            # if soft 17, dealer must hit
            if self.hand.value == 17:
                if self.hand.soft:
                    self.hit()
                else:
                    break
            # less than 17, must hit
            else:
                self.hit()
                      
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def showHands(self):
        print('Dealers Hand:')
        print(self.hand)

#### Test Player and Dealer

In [271]:
passed = 0

# Test constructor
player = Player()
if len(player.hands[0]) == 0:
    passed += 1
    
# Test Player.makeBet()
player.makeBet(0)
if player.money == 4950:
    if player.bets[0] == 50:
        passed += 1
        
# Test Player.hasMoney()
if player.hasMoney():
    passed += 1
player.money = 0
if not player.hasMoney():
    passed += 1
player.money = 4950
    
# Tests passed
if passed == 4:
    print('All tests passed')
else:
    print(str(4-passed) + ' tests failed')

All tests passed


In [272]:
passed = 0
player = Player()
deck = Deck()
dealer = Dealer(deck, player)

# Test instantiation
if len(dealer.deck) == 52:
    passed += 1
if len(dealer.player.hands[0]) == 0:
    passed += 1
if len(dealer.hand) == 0:
    passed += 1
    
# Test dealer.deal()
dealer.deal()
if len(dealer.player.hands[0]) == 2:
    passed += 1
if len(dealer.hand) == 2:
    passed += 1
if len(dealer.deck) == 48:
    passed += 1
    
# Test dealer.hit()
dealer.hit()
if len(dealer.hand) == 3:
    passed += 1
if len(dealer.deck) == 47:
    passed += 1
    
# Test dealer.play()
deck = Deck()
player = Player()
dealer = Dealer(deck, player)
dealer.hand.addCard(ace)
dealer.hand.addCard(six)
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
    
# Tests passed
if passed == 11:
    print('All tests passed')
else:
    print(str(11-passed) + ' tests failed')

All tests passed


## Valid Moves and Make

In [273]:
# returns a list of valid moves for a hand
def validMoves(player, handNumber):
    moves = ['hit','stand']
    # check that hand has 2 cards and the player has money to make a bet
    if len(player.hands[handNumber].cards) == 2:
        if player.hasMoney(): 
            moves.append('doubleDown')
            if player.hands[handNumber].cards[0] == player.hands[handNumber].cards[1]: # compare ranks same
                moves.append('split')
            
    return moves
    
def makeMove(game, handNumber, move):
    if move == 'hit':
        game.player.hit(handNumber, game.deck.draw())
        return False
    elif move == 'split':
        game.player.split(handNumber, [game.deck.draw(), game.deck.draw()])
        return False
    elif move == 'doubleDown':
        game.player.doubleDown(handNumber, game.deck.draw())
        return True
    else:
        return True
        


#### Testing ValidMoves and MakeMove

In [274]:
# Testing valid Moves
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(king)

# check all moves are avilable
moves = validMoves(player, 0)
if moves == ['hit','stand','doubleDown','split']:
    print('All moves valid: passed')
else:
    print('All moves valid: failed')
    print(moves)
    
# check no money moves
player = Player(0)
player.hands[0].addCard(king)
player.hands[0].addCard(king)
moves = validMoves(player, 0)
if moves == ['hit','stand']:
    print('No Money moves valid: passed')
else:
    print('No Money moves valid: failed')
    print(moves)
    
# check no split moves
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(five)
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown']:
    print('No split moves valid: passed')
else:
    print('No split moves valid: failed')
    print(moves)
    
# check basics - more than 2 cards
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(five)
player.hands[0].addCard(four)
moves = validMoves(player, 0)
if moves == ['hit','stand']:
    print('Basic moves 3 cards valid: passed')
else:
    print('Basic moves 3 valid: failed')
    print(moves)
    
# check split moves
player = Player()
player.hands[0].addCard(five)
player.hands[0].addCard(five)
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown', 'split']:
    print('Split moves valid: passed')
else:
    print('Split moves valid: failed')
    print(moves)

All moves valid: passed
No Money moves valid: passed
No split moves valid: passed
Basic moves 3 cards valid: passed
Split moves valid: passed


In [275]:
player = Player()

# Test Split
player.hands[0].addCard(king)
player.hands[0].addCard(king)
player.bets.append(player.betAmount)
player.split(0, [five, two])
if player.hands[0].cards == [king, five]:   
    if player.hands[1].cards == [king, two]:
        if player.bets == [50,50]:
            print('split move: passed')
        else:
            print('split move: failed')
    else:
        print('split move: failed')
else:
    print('split move: failed')
    
# Test doubleDown
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(two)
player.makeBet(0)
player.doubleDown(0, ten)
if player.hands[0].cards == [king, two, ten]:
    if player.bets == [100]:
        print('doubleDown move: passed')
    else:
        print('doubleDown move: failed')
else:
    print('doubleDown move: failed')
    
# Test hit
player.hit(0, ace)
if player.hands[0].cards == [king, two, ten, ace]:
    print('hit move: passed')
else:
    print('hit move: failed')    

split move: passed
doubleDown move: passed
hit move: passed


## Blackjack Game

In [276]:
class Blackjack:
    
    def __init__(self, Player):
        self.player = Player
        self.deck = Deck(6)
        self.dealer = Dealer(self.deck, player) # Remember, dealer holds the deck and deals, not the game
        self.dealer.deal()
        
    def newHand(self):
        self.player.hands = [Hand()]
        self.player.bets = []
        self.dealer.hand = Hand()
        self.dealer.deal()
        
    def newDeck(self):
        self.dealer.deck = Deck(6)
       
    # hand is player's hand
    def gameStatus(self, hand):
        # player bust
        if hand.checkBust():
            return -1
        # dealer bust
        if self.Dealer.hand.checkBust():
            return 1
        # draw
        if hand.value == self.Dealer.hand.value:
            return 0
        # player win
        if hand.value > self.Dealer.hand.value:
            return 1
        # dealer win
        if hand.value < self.Dealer.hand.value:
            return -1
        

#### Test Blackjack

In [277]:
player = Player()
game = Blackjack(player)
passed = 0

def testBlackjack(game, decksize, money):
    passed = 0
    # Test player dealt correct number of cards, makes bet, and loses money coorectly
    if len(game.player.hands) == 1:
        if len(game.player.hands[0].cards) == 2:
            passed += 1
    if len(game.player.bets) == 1:
        if game.player.bets[0] == 50:
            passed += 1
    if game.player.money == money:
        passed += 1
    # Test dealer dealt correct number of cards
    if len(game.dealer.hand.cards) == 2:
        passed += 1
    # Test deck dealt correctly
    if len(game.dealer.deck) == decksize:
        passed += 1
    return passed

# Test initialization
passed += testBlackjack(game, 308, 4950)
# Test new hand
game.newHand()
passed += testBlackjack(game, 304, 4900)
# Test new deck
game.newDeck()
passed += testBlackjack(game, 312, 4900)

# Tests passed
if passed == 15:
    print('All tests passed')
else:
    print(str(15-passed) + ' tests failed')

All tests passed


## TrainQ

In [278]:
import random

# determines if a greedy move should be taken
def epsilonGreedy (epsilon, Q, player, handNumber, validMovesF):
    validMoves = validMovesF(player, handNumber)
    
    if np.random.uniform() < epsilon: # Random choice
        return random.choice(validMoves)
    else: # Greedy choice
        # Please Stand By - In Progress
        return 0

In [279]:
import numpy as np

def trainQ(nRepetitions, learningRate, epsilonDecayRate, validMovesF, makeMoveF):
    epsilon = 1.0
    
    Q = {}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    
    for gameNum in range(nRepetitions):
        epsilon *= epsilonDecayRate  # decay epsilon to move away from random choices
        epsilons[gameNum] = epsilon
        
        # create a game
        player = Player()
        blackjack = Blackjack(player)
        
        done = False
        
        # play some blackjack
        while len(blackjack.Deck) >  52: # deck is 6 decks, last deck is a buffer deck
            if blackjack.player.hasMoney(): # make sure player has the funds to play
                blackjack.newHand() # deal a hand
                handNumber = 0
            
                # For each players hand
                while handNumber < len(blackjack.player.hands):
                    done = False
                
                    # Player plays until they stand or bust
                    while not done:
                        # Determine a move for the players hand
                        move = epsilonGreedy(epsilon, Q, blackjack.player, handNumber, validMovesF)
                        
                        # make a move on the hand, done if double down or stand
                        done = makeMoveF(blackjack, handNumber, move)
                        
                        # check if bust
                        if blackjack.player.hands[handNumber].checkBust():
                            done = True
                            
                    handNumber += 1 # next hand
            else:
                break # player doesn't have the funds, game over 
        
        ### After Player plays all his hands dealer plays
        ### After Dealer plays all player hands are checked
            # payout on wins
            
        
    
    # need to add Q table stuff
        # whats my key? my cards or total value of cards?
        # how will i keep track of hands?
            # list of moves
            # each hand that leads to a win or loss or draw gets a value
    # greedy choice
    # payout on wins (game)
    
    # re-test valid moves
        # updated to include player has the funds required

## Testing

In [280]:
deck = Deck(1)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

52
['Q', 'A', '2', '3', '4', 'J', '8', '9', 'A', 'K', '8', '9', '4', '7', '10', '5', '9', 'J', '3', '4', '6', '2', '5', 'K', '4', '6', '10', '3', 'A', '8', '9', '8', '5', 'K', '7', 'Q', 'J', '2', '5', '10', '10', 'Q', '7', '3', '6', '7', 'J', 'Q', '2', 'K', 'A', '6']
Q
A
50


In [281]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.value))

156
['3']
['3', '8']
['3', '8', '5']
16


In [282]:
player = Player()
game = Blackjack(player)
print(len(game.Deck))
game.Player.showHands()
game.Dealer.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.showHands()
game.newHand()
print(len(game.Deck))
game.Player.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.hit()
game.Dealer.showHands()
game.newDeck()
print(len(game.Deck))

AttributeError: 'Blackjack' object has no attribute 'Deck'

In [ ]:
game.gameStatus(game.Player.hands[0])

#### Testing validMoves and makeMove